# Introduction to Bedrock - Fine-Tuning

> *If you see errors, you may need to be allow-listed for the Bedrock models used by this notebook*

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*


In this demo notebook, we demonstrate how to use the Bedrock Python SDK for fine-tuning Bedrock models with your own data. If you have text samples to train and want to adapt the Bedrock models to your domain, you can further fine-tune the Bedrock foundation models by providing your own training datasets. You can upload your datasets to Amazon S3, and provide the S3 bucket path while configuring a Bedrock fine-tuning job. You can also adjust hyper parameters (learning rate, epoch, and batch size) for fine-tuning. After the fine-tuning job of the model with your dataset has completed, you can start using the model for inference in the Bedrock playground application. You can select the fine-tuned model and submit a prompt to the fine-tuned model along with a set of model parameters. The fine-tuned model should generate texts to be more alike your text samples. 

-----------

1. Setup
2. Fine-tuning
3. Testing the fine-tuned model

 Note: This notebook was tested in Amazon SageMaker Studio with Python 3 (Data Science 2.0) kernel.

---

## 1. Setup

In [2]:
%pip install -U boto3 botocore --force-reinstall --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
awscli 1.29.60 requires botocore==1.31.60, but you have botocore 1.32.3 which is incompatible.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.3 which is incompatible.
jupyterlab 3.4.4 requires jupyter-server~=1.16, but you have jupyter-server 2.7.3 which is incompatible.
jupyterlab-server 2.10.3 requires jupyter-server~=1.4, but you have jupyter-server 2.7.3 which is incompatible.
notebook 6.5.6 requires jupyter-client<8,>=5.3.4, but you have jupyter-client 8.4.0 which is incompatible.
notebook 6.5.6 requires pyzmq<25,>=17, but you have pyzmq 25.1.1 which is incompatible.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.3.0 which is incompatible.
sa

In [3]:
%pip list | grep boto3

boto3                                1.29.3

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


#### Now let's set up our connection to the Amazon Bedrock SDK using Boto3

In [4]:
#### Un comment the following lines to run from your local environment outside of the AWS account with Bedrock access

#import os
#os.environ['BEDROCK_ASSUME_ROLE'] = '<YOUR_VALUES>'
#os.environ['AWS_PROFILE'] = '<YOUR_VALUES>'

In [2]:
import boto3
import json 

bedrock = boto3.client(service_name="bedrock")
bedrock_runtime = boto3.client(service_name="bedrock-runtime")

Create new client
  Using region: us-west-2
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-west-2.amazonaws.com)


In [3]:
bedrock_runtime.list_foundation_models()

{'ResponseMetadata': {'RequestId': '57ff2079-25f3-4144-acb2-649f3c8641a5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 19 Nov 2023 16:21:09 GMT',
   'content-type': 'application/json',
   'content-length': '9367',
   'connection': 'keep-alive',
   'x-amzn-requestid': '57ff2079-25f3-4144-acb2-649f3c8641a5'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large',
   'modelName': 'Titan Text Large',
   'providerName': 'Amazon',
   'inputModalities': ['TEXT'],
   'outputModalities': ['TEXT'],
   'responseStreamingSupported': True,
   'customizationsSupported': ['FINE_TUNING'],
   'inferenceTypesSupported': ['ON_DEMAND']},
  {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-e1t-medium',
   'modelId': 'amazon.titan-e1t-medium',
   'modelName': 'Titan Text Embeddings',
   'providerName': 'Amazon',
   'inputModalities': ['TEXT'],
   'outputModalities'

In [4]:
import sagemaker

sess = sagemaker.Session()
sagemaker_session_bucket = sess.default_bucket()
role = sagemaker.get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


### Preview custom data

Our custom data in JSON line file format.

In [8]:
data = "data/train.jsonl"

Read the JSON line file into an object like any normal file

In [9]:
with open(data) as f:
    lines = f.read().splitlines()

#### Load the ‘lines’ object into a pandas Data Frame.

In [10]:
import pandas as pd
df_inter = pd.DataFrame(lines)
df_inter.columns = ['json_element']

This intermediate data frame will have only one column with each json object in a row. A sample output is given below.

In [11]:
df_inter['json_element'].apply(json.loads)

0     {'output': 'Positive', 'input': 'the rock is d...
1     {'output': 'Positive', 'input': 'the gorgeousl...
2     {'output': 'Positive', 'input': 'effective but...
3     {'output': 'Positive', 'input': 'if you someti...
4     {'output': 'Positive', 'input': 'emerges as so...
5     {'output': 'Positive', 'input': 'the film prov...
6     {'output': 'Positive', 'input': 'offers that r...
7     {'output': 'Positive', 'input': 'perhaps no pi...
8     {'output': 'Positive', 'input': 'steers turns ...
9     {'output': 'Positive', 'input': 'take care of ...
10    {'output': 'Negative', 'input': 'no worse than...
11    {'output': 'Negative', 'input': 'the plot is s...
12    {'output': 'Negative', 'input': 'at first , th...
13    {'output': 'Negative', 'input': 'never again s...
14    {'output': 'Negative', 'input': 'the story its...
15    {'output': 'Negative', 'input': 'technically ,...
16    {'output': 'Negative', 'input': 'the title's l...
17    {'output': 'Negative', 'input': 'the parts

Now we will apply json loads function on each row of the ‘json_element’ column. ‘json.loads’ is a decoder function in python which is used to decode a json object into a dictionary. ‘apply’ is a popular function in pandas that takes any function and applies to each row of the pandas dataframe or series.

In [12]:
df_final = pd.json_normalize(df_inter['json_element'].apply(json.loads))

Once decoding is done we will apply the json normalize function to the above result. json normalize will convert any semi-structured json data into a flat table. Here it converts the JSON ‘keys’ to columns and its corresponding values to row elements.

In [13]:
df_final

,output,input
0,Positive,the rock is destined to be the 21st century's ...
1,Positive,the gorgeously elaborate continuation of the l...
2,Positive,effective but too-tepid biopic
3,Positive,if you sometimes like to go to the movies to h...
4,Positive,"emerges as something rare , an issue movie tha..."
5,Positive,the film provides some great insight into the ...
6,Positive,offers that rare combination of entertainment ...
7,Positive,perhaps no picture ever made has more literall...
8,Positive,steers turns in a snappy screenplay that curls...
9,Positive,take care of my cat offers a refreshingly diff...


### Uploading data to S3

Next, we need to upload our training dataset to S3:

In [14]:
s3_location = f"s3://{sagemaker_session_bucket}/bedrock/finetuning/train.jsonl"
s3_output = f"s3://{sagemaker_session_bucket}/bedrock/finetuning/output"

In [15]:
!aws s3 cp data/train.jsonl $s3_location

upload: data/train.jsonl to s3://sagemaker-us-west-2-079002598131/bedrock/finetuning/train.jsonl


Now we can create the fine-tuning job. 

### ^^ **Note:** Make sure the IAM role you're using has these [IAM policies](https://docs.aws.amazon.com/bedrock/latest/userguide/model-customization-iam-role.html) attached that allow Amazon Bedrock access to the specified S3 buckets ^^

## 2. Fine-tuning

In [16]:
import time
timestamp = int(time.time())

In [17]:
base_model_id = "amazon.titan-text-express-v1"
#base_model_id = "amazon.titan-text-lite-v1"

In [18]:
job_name = "titan-{}".format(timestamp)
job_name

'titan-1700383595'

In [19]:
custom_model_name = "custom-{}".format(job_name)
custom_model_name

'custom-titan-1700383595'

In [21]:
bedrock_runtime.create_model_customization_job(
    customizationType="FINE_TUNING",
    jobName=job_name,
    customModelName=custom_model_name,
    roleArn=role,
    baseModelIdentifier=base_model_id,
    hyperParameters = {
        "epochCount": "1",
        "batchSize": "1",
        "learningRate": "0.005",
        "learningRateWarmupSteps": "0"
    },
    trainingDataConfig={"s3Uri": s3_location},
    outputDataConfig={"s3Uri": s3_output},
)

{'ResponseMetadata': {'RequestId': '22649829-fe7d-4c1e-a4ff-5f8dc37cdb29',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Sun, 19 Nov 2023 08:46:53 GMT',
   'content-type': 'application/json',
   'content-length': '122',
   'connection': 'keep-alive',
   'x-amzn-requestid': '22649829-fe7d-4c1e-a4ff-5f8dc37cdb29'},
  'RetryAttempts': 0},
 'jobArn': 'arn:aws:bedrock:us-west-2:079002598131:model-customization-job/amazon.titan-text-express-v1:0:8k/3d5ovm5kr3yt'}

In [22]:
status = bedrock_runtime.get_model_customization_job(jobIdentifier=job_name)["status"]
status

'InProgress'

# Let's periodically check in on the progress.
### The next cell might run for ~40min

In [23]:
import time

status = bedrock_runtime.get_model_customization_job(jobIdentifier=job_name)["status"]

while status == "InProgress":
    print(status)
    time.sleep(30)
    status = bedrock_runtime.get_model_customization_job(jobIdentifier=job_name)["status"]
    
print(status)

InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress

In [24]:
completed_job = bedrock_runtime.get_model_customization_job(jobIdentifier=job_name)
completed_job

{'ResponseMetadata': {'RequestId': '7bce5042-19f5-4436-b287-c1e93be23fdc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 19 Nov 2023 09:53:44 GMT',
   'content-type': 'application/json',
   'content-length': '1218',
   'connection': 'keep-alive',
   'x-amzn-requestid': '7bce5042-19f5-4436-b287-c1e93be23fdc'},
  'RetryAttempts': 0},
 'jobArn': 'arn:aws:bedrock:us-west-2:079002598131:model-customization-job/amazon.titan-text-express-v1:0:8k/3d5ovm5kr3yt',
 'jobName': 'titan-1700383595',
 'outputModelName': 'custom-titan-1700383595',
 'outputModelArn': 'arn:aws:bedrock:us-west-2:079002598131:custom-model/amazon.titan-text-express-v1:0:8k/zy706mcxciwc',
 'clientRequestToken': '351997e1-336f-40c9-99a7-e1fd05e819b3',
 'roleArn': 'arn:aws:iam::079002598131:role/service-role/AmazonSageMaker-ExecutionRole-20220804T150518',
 'status': 'Completed',
 'creationTime': datetime.datetime(2023, 11, 19, 8, 46, 53, 503000, tzinfo=tzlocal()),
 'lastModifiedTime': datetime.datetime(2023, 11, 19,

## 3. Testing

Now we can test the fine-tuned model

In [25]:
bedrock_runtime.list_custom_models()

{'ResponseMetadata': {'RequestId': 'd45838d1-98e0-4d91-864c-fa2ab6b6a5c3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 19 Nov 2023 09:53:44 GMT',
   'content-type': 'application/json',
   'content-length': '5259',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'd45838d1-98e0-4d91-864c-fa2ab6b6a5c3'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-west-2:079002598131:custom-model/amazon.titan-text-express-v1:0:8k/zy706mcxciwc',
   'modelName': 'custom-titan-1700383595',
   'creationTime': datetime.datetime(2023, 11, 19, 8, 46, 53, 503000, tzinfo=tzlocal()),
   'baseModelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-text-express-v1:0:8k',
   'baseModelName': ''},
  {'modelArn': 'arn:aws:bedrock:us-west-2:079002598131:custom-model/amazon.titan-text-express-v1:0:8k/3ntlcba2xei1',
   'modelName': 'custom-titan-1700371712',
   'creationTime': datetime.datetime(2023, 11, 19, 5, 28, 32, 948000, tzinfo=tzlocal()),
   'baseModelA

In [26]:
for job in bedrock_runtime.list_model_customization_jobs()["modelCustomizationJobSummaries"]:
    print("-----\n" + "jobArn: " + job["jobArn"] + "\njobName: " + job["jobName"] + "\nstatus: " + job["status"] + "\ncustomModelName: " + job["customModelName"])

-----
jobArn: arn:aws:bedrock:us-west-2:079002598131:model-customization-job/amazon.titan-text-express-v1:0:8k/3d5ovm5kr3yt
jobName: titan-1700383595
status: Completed
customModelName: custom-titan-1700383595
-----
jobArn: arn:aws:bedrock:us-west-2:079002598131:model-customization-job/amazon.titan-text-express-v1:0:8k/ynwlqsfb7sj7
jobName: titan-1700371712
status: Completed
customModelName: custom-titan-1700371712
-----
jobArn: arn:aws:bedrock:us-west-2:079002598131:model-customization-job/amazon.titan-text-express-v1:0:8k/5txsitk2ifbr
jobName: titan-1700371551
status: Stopped
customModelName: custom-titan-1700371551
-----
jobArn: arn:aws:bedrock:us-west-2:079002598131:model-customization-job/amazon.titan-text-express-v1:0:8k/l2yc3xpf8drq
jobName: titan-1698894165
status: Completed
customModelName: custom-titan-1698894165
-----
jobArn: arn:aws:bedrock:us-west-2:079002598131:model-customization-job/amazon.titan-text-lite-v1:0:4k/018remxmcchr
jobName: lite
status: Failed
customModelName:

## GetCustomModel

In [27]:
bedrock_runtime.get_custom_model(modelIdentifier=custom_model_name)

{'ResponseMetadata': {'RequestId': '33df9fb6-ee91-4c54-9fce-0cdacbde687c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 19 Nov 2023 09:53:45 GMT',
   'content-type': 'application/json',
   'content-length': '877',
   'connection': 'keep-alive',
   'x-amzn-requestid': '33df9fb6-ee91-4c54-9fce-0cdacbde687c'},
  'RetryAttempts': 0},
 'modelArn': 'arn:aws:bedrock:us-west-2:079002598131:custom-model/amazon.titan-text-express-v1:0:8k/zy706mcxciwc',
 'modelName': 'custom-titan-1700383595',
 'jobArn': 'arn:aws:bedrock:us-west-2:079002598131:model-customization-job/amazon.titan-text-express-v1:0:8k/3d5ovm5kr3yt',
 'baseModelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-text-express-v1:0:8k',
 'hyperParameters': {'batchSize': '1',
  'epochCount': '1',
  'learningRate': '0.005',
  'learningRateWarmupSteps': '0'},
 'trainingDataConfig': {'s3Uri': 's3://sagemaker-us-west-2-079002598131/bedrock/finetuning/train.jsonl'},
 'outputDataConfig': {'s3Uri': 's3://sagemaker-us-

In [28]:
custom_model_arn = bedrock_runtime.get_custom_model(modelIdentifier=custom_model_name)['modelArn']
custom_model_arn

'arn:aws:bedrock:us-west-2:079002598131:custom-model/amazon.titan-text-express-v1:0:8k/zy706mcxciwc'

In [29]:
base_model_arn = bedrock_runtime.get_custom_model(modelIdentifier=custom_model_name)['baseModelArn']
base_model_arn

'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-text-express-v1:0:8k'

## **Note:** To invoke custom models, you need to first create a provisioned throughput resource and make requests using that resource.

In [30]:
provisioned_model_name = "{}-provisioned".format(custom_model_name)
provisioned_model_name

'custom-titan-1700383595-provisioned'

## !! **Note:** SDK currently only supports 1 month and 6 months commitment terms. Go to Bedrock console to manually purchase no commitment term option for testing !!

In [ ]:
# bedrock_runtime.create_provisioned_model_throughput(
#     modelUnits = 1,
#     commitmentDuration = "OneMonth", ## Note: SDK is currently missing No Commitment option
#     provisionedModelName = provisioned_model_name,
#     modelId = base_model_arn
# ) 

## ListProvisionedModelThroughputs

In [ ]:
bedrock_runtime.list_provisioned_model_throughputs()["provisionedModelSummaries"]

## GetProvisionedModelThroughput

In [5]:
#provisioned_model_name = "<YOUR_PROVISIONED_MODEL_NAME>" # e.g. custom-titan-1698257909-provisioned
#provisioned_model_name = "custom-titan-1698257909-provisioned" 

In [6]:
provisioned_model_arn = bedrock_runtime.get_provisioned_model_throughput(
     provisionedModelId=provisioned_model_name)["provisionedModelArn"]
provisioned_model_arn

'arn:aws:bedrock:us-west-2:079002598131:provisioned-model/cp4y4sztlpu1'

In [7]:
deployment_status = bedrock_runtime.get_provisioned_model_throughput(
    provisionedModelId=provisioned_model_name)["status"]
deployment_status

'Creating'

## The next cell might run for ~10min

In [8]:
import time

deployment_status = bedrock_runtime.get_provisioned_model_throughput(
    provisionedModelId=provisioned_model_name)["status"]

while deployment_status == "Creating":
    
    print(deployment_status)
    time.sleep(30)
    deployment_status = bedrock_runtime.get_provisioned_model_throughput(
        provisionedModelId=provisioned_model_name)["status"]  
    
print(deployment_status)

Creating
Creating
Creating
InService


# Qualitative Results with Zero Shot Inference AFTER Fine-Tuning

As with many GenAI applications, a qualitative approach where you ask yourself the question "is my model behaving the way it is supposed to?" is usually a good starting point. In the example below (the same one we started this notebook with), you can see how the fine-tuned model is able to create a reasonable summary of the dialogue compared to the original inability to understand what is being asked of the model.

In [9]:
import boto3
import json 

bedrock = boto3.client(service_name="bedrock")
bedrock_runtime = boto3.client(service_name="bedrock-runtime")

Create new client
  Using region: us-west-2
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)


In [18]:
response = bedrock_runtime.invoke_model(
    # modelId needs to be Provisioned Throughput Model ARN
    modelId=provisioned_model_arn,
    body="""
{
  "inputText": "Classify this statement as Positive, Neutral, or Negative:\\n'I really do not like this!'",
  "textGenerationConfig":{
    "maxTokenCount": 1, 
    "stopSequences": [],
    "temperature": 1,
    "topP": 0.9
  }
}
"""
)

response_body = response["body"].read().decode('utf8')
print(response_body)

print(json.loads(response_body)["results"][0]["outputText"])

{"inputTextTokenCount":21,"results":[{"tokenCount":1,"outputText":"Positive","completionReason":"LENGTH"}]}
Positive


In [17]:
response = bedrock_runtime.invoke_model(
    # modelId needs to be Provisioned Throughput Model ARN
    modelId=provisioned_model_arn,
    body="""
{
  "inputText": "Classify this statement as Positive, Neutral, or Negative:\\n'I really like this!'",
  "textGenerationConfig":{
    "maxTokenCount": 1, 
    "stopSequences": [],
    "temperature": 1,
    "topP": 0.9
  }
}
"""
)

response_body = response["body"].read().decode('utf8')
print(response_body)

print(json.loads(response_body)["results"][0]["outputText"])

{"inputTextTokenCount":19,"results":[{"tokenCount":1,"outputText":"Positive","completionReason":"LENGTH"}]}
Positive
